<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Daralan_Bollingerbant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# TradingView API Bağlantısı
tv = TvDatafeed()

# Türkiye piyasasındaki hisseleri alma
try:
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
except Exception as e:
    print(f"Hata alındı: {e}")

# Zaman dilimi seçimi için fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen tarama için bir zaman dilimi seçin:")
    for key, (name, _) in intervals.items():
        print(f"{key}: {name}")

    while True:
        choice = input("Seçiminizi yapın (1-6): ")
        if choice in intervals:
            return intervals[choice][1]
        else:
            print("Geçersiz seçim, lütfen tekrar deneyin.")

# Daralan Bollinger Bandı Taraması
def bollinger_squeeze(data, period=20, threshold=0.02):
    # Bollinger Bands hesaplama
    middle_band = data['close'].rolling(window=period).mean()
    std_dev = data['close'].rolling(window=period).std()
    upper_band = middle_band + 2 * std_dev
    lower_band = middle_band - 2 * std_dev

    # Bollinger bandı genişliği
    data['bb_width'] = (upper_band - lower_band) / middle_band

    # Daralma tespiti
    data['squeeze'] = data['bb_width'] < threshold
    return data[data['squeeze']]  # Sıkışma durumlarını döndür

# Hisse tarama fonksiyonu
def scan_bollinger_squeeze(symbols, interval, n_bars=500, bb_period=20, bb_threshold=0.02):
    results = []

    for symbol in symbols:
        try:
            # TradingView'den veri çekme
            data = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=n_bars)
            if data is None or data.empty:
                print(f"{symbol} için veri bulunamadı.")
                continue

            # Daralan Bollinger Bandı taraması
            squeezed_data = bollinger_squeeze(data, period=bb_period, threshold=bb_threshold)

            if not squeezed_data.empty:
                last_squeeze = squeezed_data.iloc[-1]
                results.append({
                    'Hisse Adı': symbol,
                    'Son Fiyat': last_squeeze['close'],
                    'Sıkışma Tarihi': last_squeeze.name,
                    'BB Genişlik': last_squeeze['bb_width']
                })
                print(f"{symbol}: Sıkışma tespit edildi ({last_squeeze.name.date()})")
        except Exception as e:
            print(f"{symbol} için hata: {e}")

    return results

# Kullanıcıdan zaman dilimi seçimi
interval = get_interval_choice()

# Taramayı başlatma
results = scan_bollinger_squeeze(Hisseler, interval)

# Sonuçları gösterme
if results:
    df_results = pd.DataFrame(results)
    print("\nDaralan Bollinger Bandı Tespit Edilen Hisseler:")
    print(df_results)
else:
    print("\nHiçbir hisse için Bollinger Bandı sıkışması tespit edilmedi.")



  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-_hjukd4h
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-_hjukd4h
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=7cf1ec3288bdd44b94a066143596c5492c53ea25421fa5d625974f5b104b5646
  Stored in directory: /tmp/pip-ephem-wheel-cache-5jkkdrf8/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 665.8 kB/s eta 0:00:00


Lütfen tarama için bir zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
ALTIN: Sıkışma tespit edildi (2024-02-28)
GLDTR: Sıkışma tespit edildi (2024-02-19)
USDTR: Sıkışma tespit edildi (2025-01-21)
YGGYO: Sıkışma tespit edildi (2023-12-01)
ZGOLD: Sıkışma tespit edildi (2023-08-22)

Daralan Bollinger Bandı Tespit Edilen Hisseler:
  Hisse Adı    Son Fiyat      Sıkışma Tarihi  BB Genişlik
0     ALTIN    23.309999 2024-02-28 06:00:00     0.019692
1     GLDTR   177.800000 2024-02-19 06:00:00     0.019546
2     USDTR  3340.000000 2025-01-21 06:00:00     0.016065
3     YGGYO    39.000000 2023-12-01 06:00:00     0.013152
4     ZGOLD   184.800000 2023-08-22 06:00:00     0.019729
